In [1]:
import pandas as pd

# Create a Machine Learning-friendly dataset

In [3]:
# Import the master data file into a dataframe
master_df = pd.read_csv('masterFinalData11_24_2020.csv')

In [4]:
print(master_df.columns)
master_df.head()

Index(['Unnamed: 0', 'gameID', 'statsID', 'playerID', 'playerFullName',
       'playerFirstName', 'playerLastName', 'playerPosition', 'age',
       'playerHeightFeet', 'playerHeightInches', 'playerWeightPounds',
       'assists', 'blocks', 'defensiveRebounds', 'threePointFieldGoalPercent',
       'threePointFieldGoalAttempted', 'threePointFieldGoal',
       'fieldGoalPercent', 'fieldGoalAttempt', 'fieldGoalMade',
       'freeThrowPercent', 'freeThrowAttempt', 'freeThrowMade', 'minutes',
       'offensiveRebound', 'personalFouls', 'points', 'rebounds', 'steals',
       'teamID', 'turnovers', 'winner_x', 'date', 'homeTeamScore',
       'visitorTeamScore', 'season_y', 'time', 'homeTeamID', 'homeTeamAbbrev',
       'homeTeamCity', 'homeTeamName', 'visitorTeamID', 'visitorTeamAbbrev',
       'visitorTeamCity', 'visitorTeamName', 'team_abbreviation',
       'playerHeight', 'playerWeight', 'college', 'country', 'draft_year',
       'draft_round', 'draft_number', 'gp', 'pts', 'reb', 'ast', 'ne

,Unnamed: 0,gameID,statsID,playerID,playerFullName,playerFirstName,playerLastName,playerPosition,age,playerHeightFeet,...,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct
0,0,27276,708256,364,Chandler Parsons,Chandler,Parsons,F,26.0,6.0,...,66.0,15.7,4.9,2.4,4.6,0.032,0.132,0.205,0.567,0.111
1,1,27276,708257,346,Dirk Nowitzki,Dirk,Nowitzki,F,37.0,7.0,...,77.0,17.3,5.9,1.9,4.3,0.021,0.206,0.251,0.560,0.101
2,2,27276,708258,94,Tyson Chandler,Tyson,Chandler,C,32.0,7.0,...,75.0,10.3,11.5,1.1,5.5,0.141,0.271,0.128,0.697,0.053
3,3,27276,708259,1635,Monta Ellis,Monta,Ellis,NaN,29.0,NaN,...,80.0,18.9,2.4,4.1,4.2,0.014,0.063,0.277,0.509,0.200
4,4,27276,708260,1533,Jameer Nelson,Jameer,Nelson,NaN,33.0,NaN,...,63.0,8.3,2.3,4.0,-0.6,0.025,0.091,0.199,0.503,0.278


In [5]:
# Create the empty dataframe that will be used for machine learning
columns = ['gameID', 'homeTeamID', 'visitorTeamID', 'homeTeamHeightAverage', 'homeTeamWeightAverage', 'homeTeamAgeAverage', 'visitorTeamHeightAverage', 'visitorTeamWeightAverage', 'visitorTeamAgeAverage', 'homeTeamWin']
ml_df = pd.DataFrame(columns=columns)
ml_df.head()



,gameID,homeTeamID,visitorTeamID,homeTeamHeightAverage,homeTeamWeightAverage,homeTeamAgeAverage,visitorTeamHeightAverage,visitorTeamWeightAverage,visitorTeamAgeAverage,homeTeamWin


In [6]:
# Get rid of unneeded columns
master_clean_df = master_df[['gameID', 'playerID', 'teamID', 'age', 'playerHeight', 'playerWeight', 'homeTeamScore', 
                             'visitorTeamScore', 'homeTeamID', 'visitorTeamID',  'winner_x',]]
master_clean_df.head()

,gameID,playerID,teamID,age,playerHeight,playerWeight,homeTeamScore,visitorTeamScore,homeTeamID,visitorTeamID,winner_x
0,27276,364,7,26.0,205.74,102.965384,108,87,25,7,0
1,27276,346,7,37.0,213.36,111.130040,108,87,25,7,0
2,27276,94,7,32.0,215.90,108.862080,108,87,25,7,0
3,27276,1635,7,29.0,190.50,83.914520,108,87,25,7,0
4,27276,1533,7,33.0,182.88,86.182480,108,87,25,7,0


In [7]:
# Group by game ID
group_by_gameID = master_clean_df.groupby(['gameID'])
group_by_gameID

In [8]:
# Iterate over the games
for name, group in group_by_gameID:
    # gather the data we need to fill our dataframe
    print(name)
    gameID = name
    homeTeamID = group['homeTeamID'].values[0]
    visitorTeamID = group['visitorTeamID'].values[0]
    homeTeamScore = group['homeTeamScore'].values[0]
    visitorTeamScore = group['visitorTeamScore'].values[0]

    # group by the team to get team average values
    groupByTeamsHomeAvgs = group.groupby(['teamID']).get_group(homeTeamID).mean()
    groupByTeamsVisAvgs = group.groupby(['teamID']).get_group(visitorTeamID).mean()
    homeHeightAvg = groupByTeamsHomeAvgs['playerHeight']
    homeWeightAvg = groupByTeamsHomeAvgs['playerWeight']
    homeAgeAvg = groupByTeamsHomeAvgs['age']
    visHeightAvg = groupByTeamsVisAvgs['playerHeight']
    visWeightAvg = groupByTeamsVisAvgs['playerWeight']
    visAgeAvg = groupByTeamsVisAvgs['age']

    # did home team win? 1 else 0
    homeTeamWin = 1 if (homeTeamScore - visitorTeamScore) > 0 else 0

    # Fill the dataframe row!
    dictRow = { 'gameID': gameID, 'homeTeamID': homeTeamID, 'visitorTeamID': visitorTeamID, 
               'homeTeamHeightAverage': homeHeightAvg, 'homeTeamWeightAverage': homeWeightAvg, 
               'homeTeamAgeAverage': homeAgeAvg, 'visitorTeamHeightAverage': visHeightAvg, 
               'visitorTeamWeightAverage': visWeightAvg, 'visitorTeamAgeAverage': visAgeAvg, 
               'homeTeamWin': homeTeamWin }

    # add row to df
    ml_df = ml_df.append(dictRow, ignore_index=True)
    
ml_df.head()
        
        

        

        
    
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


29328
29329
29330
29331
29332
29333
29334
29335
29336
29337
29338
29339
29340
29341
29342
29343
29344
29345
29346
29347
29348
29395
29396
29397
29398
29399
29400
29401
29402
29403
29404
29405
29406
29407
29408
29409
29410
29411
29412
29413
29414
29415
29416
29417
29418
29419
29420
29421
29422
29423
29424
29425
29426
29427
29428
29429
29430
29431
29432
29433
29434
29435
29436
29437
29438
29439
29440
29441
29442
29443
29444
29445
29446
29447
29448
29449
29450
29451
29452
29453
29454
29455
29456
29457
29458
29459
29460
29461
29462
29463
29464
29465
29466
29467
29468
29469
29470
29471
29472
29473
29474
29475
29476
29477
29478
29479
29480
29481
29482
29483
29598
29599
29600
29601
29602
29603
29604
29605
29606
29607
29608
29609
29610
29611
29612
29613
29614
29615
29616
29617
29618
29619
29679
29680
29681
29682
29683
29684
29685
29686
29687
29688
29689
29690
29691
29692
29693
29694
29695
29696
29697
29698
29699
29700
29701
29702
29703
29704
29705
29706
29707
29708
29709
29710
29711
29712
2971

32595
32596
32597
32598
32599
32600
32601
32602
32603
32604
32605
32606
32607
32608
32609
32610
32645
32646
32647
32648
32649
32650
32651
32652
32653
32654
32655
32656
32657
32658
32659
32660
32661
32662
32663
32664
32665
32666
32667
32668
32669
32670
32671
32672
32673
32674
32675
32676
32677
32678
32679
32680
32681
32682
32683
32684
32685
32686
32687
32688
32689
32690
32691
32692
32693
32694
32695
32696
32697
32698
32699
32700
32701
32702
32703
32704
32705
32706
32707
32708
32709
32710
32711
32712
32713
32714
32715
32716
32717
32718
32719
32720
32721
32722
32723
32724
32725
32726
32727
32728
32729
32730
32731
32732
32733
32734
32735
32736
32737
32738
32739
32740
32741
32742
32743
32744
32745
32746
32774
32775
32776
32777
32778
32779
32780
32781
32782
32783
32784
32785
32786
32787
32788
32789
32790
32791
32792
32793
32794
32795
32796
32797
32798
32799
32800
32801
32802
32803
32804
32805
32806
32807
32808
32809
32810
32811
32812
32813
32814
32815
32816
32817
32818
32819
32820
32821
3282

34086
34087
34088
34089
34090
34091
34092
34093
34094
34095
34096
34097
34098
34099
34100
34101
34102
34103
34104
34105
34106
34107
34108
34109
34110
34111
34112
34113
34114
34115
34116
34117
34118
34119
34120
34121
34122
34123
34124
34125
34126
34127
34128
34129
34130
34131
34132
34133
34134
34135
34136
34137
34138
34139
34140
34141
34142
34143
34144
34145
34146
34147
34148
34149
34150
34151
34152
34153
34154
34155
34156
34157
34158
34159
34160
34161
34162
34163
34164
34165
34166
34167
34168
34169
34170
34171
34172
34173
34174
34175
34176
34177
34178
34179
34180
34181
34182
34183
34184
34185
34186
34187
34188
34189
34190
34191
34192
34193
34194
34195
34196
34197
34198
34199
34200
34201
34202
34203
34204
34205
34206
34207
34208
34209
34210
34211
34212
34213
34214
34215
34216
34217
34218
34219
34220
34221
34222
34223
34224
34225
34226
34227
34228
34229
34230
34231
34232
34233
34234
34235
34236
34237
34238
34239
34240
34241
34242
34243
34244
34245
34246
34247
34248
34249
34250
34251
3425

35493
35494
35495
35496
35497
35498
35499
35500
35501
35502
35503
35504
35505
35506
35507
35508
35509
35510
35511
35512
35513
35514
35515
35516
35517
35518
35519
35520
35521
35522
35523
35524
35525
35526
35527
35528
35529
35530
35531
35532
35533
35534
35535
35536
35537
35538
35539
35540
35541
35542
35543
35544
35545
35546
35547
35548
35549
35550
35551
35552
35553
35554
35555
35556
35557
35558
35559
35560
35561
35562
35563
35564
35565
35566
35567
35568
35569
35570
35571
35572
35573
35574
35575
35576
35577
35578
35579
35580
35581
35582
35583
35584
35585
35586
35587
35588
35589
35590
35591
35592
35593
35594
35595
35596
35597
35598
35599
35600
35601
35602
35603
35604
35605
35606
35607
35608
35609
35610
35611
35612
35613
35614
35615
35616
35617
35618
35619
35620
35621
35622
35623
35624
35625
35626
35627
35628
35629
35630
35631
35632
35633
35634
35635
35636
35637
35638
35639
35640
35641
35642
35643
35644
35645
35646
35647
35648
35649
45122
45123
45161
45162
45163
45164
45165
45166
45167
4517

KeyError: 19

In [ ]:
ml_df.to_csv('data/machineLearningDataSet.csv')

In [10]:
ml_df = pd.read_csv('machineLearningDataSet.csv')
ml_df.head()

,Unnamed: 0,gameID,homeTeamID,visitorTeamID,homeTeamHeightAverage,homeTeamWeightAverage,homeTeamAgeAverage,visitorTeamHeightAverage,visitorTeamWeightAverage,visitorTeamAgeAverage,homeTeamWin
0,0,1.0,2.0,23.0,199.390000,102.625190,26.416667,200.269231,96.440638,25.153846,1.0
1,1,2.0,10.0,21.0,201.441538,97.906089,28.000000,199.878462,97.696738,25.692308,1.0
2,2,3.0,4.0,17.0,201.050769,99.301756,26.846154,201.050769,100.941666,26.692308,0.0
3,3,4.0,9.0,3.0,198.966667,100.243832,27.466667,201.718333,98.656260,25.166667,1.0
4,4,5.0,12.0,15.0,199.683077,97.208255,26.461538,201.832308,99.511106,27.692308,1.0
